In [ ]:
from datetime import date, datetime, time
from typing import Optional, List
from decimal import Decimal
from itertools import chain
import sqlalchemy as sa
import sqlalchemy.orm as so
from sqlalchemy.ext.associationproxy import AssociationProxy, association_proxy
from sqlalchemy.ext.hybrid import hybrid_property

In [ ]:
# this notebook provides an example of some extended functionality that supports episode-driven complexity of CDM
# as required to assemble systemic anti-cancer therapy treatment episodes from source data

# this functionality assumes that the user has already assembled their source drug exposure records into a sensible hierarchy 
# of treatment episodes with inferences made regarding links to the disease being treated (where available)

# work describing the detailed assembly of such episodes using this compatible interface is underway and will be made 
# public once validated - see README for updates of progress

In [ ]:
from omop_alchemy.db import Base
from omop_alchemy.model.onco_ext import Episode, Episode_Event
from omop_alchemy.model.clinical.person import Person, Modifiable_Table, Condition_Occurrence, Drug_Exposure
from omop_alchemy.model.vocabulary import Concept
from omop_alchemy.conventions.concept_enumerators import ModifierFields

In [ ]:

condition_ep_join = sa.join(Condition_Occurrence, Episode_Event, 
                             sa.and_(Condition_Occurrence.condition_occurrence_id==Modifiable_Table.modifier_id,
                                     Modifiable_Table.modifier_id==Episode_Event.event_id,
                                     Episode_Event.episode_event_field_concept_id==ModifierFields.condition_occurrence_id.value))


In [ ]:
systemic_therapy = so.aliased(Drug_Exposure, flat=True)

# this subquery pulls in only episodes that have at least one drug exposure event linked
# assumption is made that only relevant systemic therapies are lined to cancer treatment 
# episodes. other drug exposures may exist in the data, but they are not linked to treatment 
# episodes if deemed irrelevant to direct cancer treatment.

# if this is not the case, the extended hemonc interface is a straightforward way to 
# implement the additional treatment filters if required

# TBC - handling of supportive meds

systemic_therapy_subquery = (
    sa.select(
        Episode.person_id,
        Episode.episode_id.label('sact_episode_id'),
        Episode.episode_parent_id,
        systemic_therapy.drug_exposure_start_date
    )
    .join(
         Episode_Event,
         Episode.episode_id==Episode_Event.episode_id
     )
    .join_from(
        Episode_Event,
        Episode_Event.event_polymorphic.of_type(systemic_therapy)
    )
    .subquery()
)

# for each systemic treatment episode, find the earliest drug delivery event available

systemic_therapy_start = (
    sa.select(
        systemic_therapy_subquery.c.person_id,
        systemic_therapy_subquery.c.sact_episode_id,
        systemic_therapy_subquery.c.episode_parent_id,
        sa.func.min(systemic_therapy_subquery.c.drug_exposure_start_date).label('sact_start')
    )
    .group_by(        
        systemic_therapy_subquery.c.person_id,        
        systemic_therapy_subquery.c.episode_parent_id,
        systemic_therapy_subquery.c.sact_episode_id
    )
    .subquery()
)


# here, we pull in only episodes that have at least one condition linked

diagnosis = so.aliased(Condition_Occurrence, flat=True)

dx_subquery = (
    sa.select(
        Episode.person_id,
        Episode.episode_id.label('dx_episode_id'),
        diagnosis.condition_start_date
    )
    .join(
         Episode_Event,
         Episode.episode_id==Episode_Event.episode_id
     )
    .join_from(
        Episode_Event,
        Episode_Event.event_polymorphic.of_type(diagnosis)
    )
    .subquery()
)

# join diagnostic episodes with earliest chemo delivery
# date in linked treatment episode to identify start of 
# systemic treatment 

systemic_therapy_with_dx = (
    sa.join(
        dx_subquery, 
        systemic_therapy_start, 
        sa.and_(
            systemic_therapy_start.c.episode_parent_id==dx_subquery.c.dx_episode_id, 
            systemic_therapy_start.c.person_id==dx_subquery.c.person_id
        ),
        isouter=True
    )
)




In [ ]:
# we then assemble the above complex queries into target mapping classes, which draw from multiple sub-queries and / or joins as required

class Condition_Episode(Base):
    __table__ = condition_ep_join
    episode_id = Episode_Event.episode_id
    person_id = Condition_Occurrence.person_id
    condition_occurrence_id = Condition_Occurrence.condition_occurrence_id  
    condition_concept_id = Condition_Occurrence.condition_concept_id
    condition_code = Condition_Occurrence.condition_code
    modifier_concepts = Condition_Occurrence.modifier_concepts
    episode_start_datetime = Episode_Event.episode_start_datetime

    person_object: so.Mapped['Person_Episodes'] = so.relationship(back_populates="condition_episodes", foreign_keys=[person_id])


class Systemic_Therapy_Episode(Base):
    __table__ = systemic_therapy_with_dx
    episode_id = systemic_therapy_start.c.sact_episode_id
    person_id = systemic_therapy_start.c.person_id
    sact_start = so.column_property(systemic_therapy_start.c.sact_start)
    
    dx_ep_id = dx_subquery.c.dx_episode_id

    dx_object: so.Mapped[Optional['Episode']] = so.relationship(foreign_keys=[dx_ep_id])
    person_object: so.Mapped['Person_Episodes'] = so.relationship(back_populates="sact_episodes", foreign_keys=[person_id])
    episode_object: so.Mapped['Episode'] = so.relationship(foreign_keys=[episode_id])
    sact_events: AssociationProxy[List['Episode_Event']] = association_proxy("episode_object", "events")

    @property
    def episode_agents(self):
        return list(set([s.event_polymorphic.drug_label for s in self.sact_events if s.event_polymorphic.polymorphic_label=='drug_exposure']))


class Person_Episodes(Person):
    condition_episodes: so.Mapped[List['Condition_Episode']] = so.relationship(back_populates="person_object", lazy="selectin")
    sact_episodes: so.Mapped[List['Systemic_Therapy_Episode']] = so.relationship(back_populates="person_object", order_by='Systemic_Therapy_Episode.sact_start')

    @property
    def all_agents(self):
        return list(set(chain.from_iterable([se.episode_agents for se in self.sact_episodes])))


In [ ]:

# this means that the end user can perform much simpler queries, such as 

import omop_alchemy as oa
engine = oa.oa_config.engine

with so.Session(engine) as session:
    person_object = session.query(Person_Episodes).filter(Person_Episodes.person_id == 1234).first()

# this will return a person object, for person_id=1234, through which all linked overarching dx episodes may 
# be accessed directly as person_object.condition_episodes, as well as any systemic therapy episodes
# as person_object.sact_episodes 

# if only the dx episodes that are linked to systemic therapy episodes are desired, one can instead 
# access [sact.dx_object for sact in person_object.sact_episodes]
